In [1]:
import os
import numpy as np
import pyvista as pv
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.Geom import Geom_Plane
from OCC.Extend.DataExchange import read_step_file
from OCC.Extend.ShapeFactory import get_oriented_boundingbox
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
pv.set_jupyter_backend('trame')  # Try interactive 3D again

!pip install reportlab

In [2]:
# Load the STEP and extract 'solid' as before
from OCC.Extend.DataExchange import read_step_file
shape = read_step_file("0444-1.step")
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
exp = TopExp_Explorer(shape, TopAbs_SOLID)
solid = exp.Current()

In [3]:
from OCC.Extend.ShapeFactory import get_oriented_boundingbox

# 1) Call the function and unpack its three returns
bary_center, half_extents, box_shape = get_oriented_boundingbox(solid)

# 2) Unpack the half-lengths
a_half_x, a_half_y, a_half_z = half_extents

# 3) Compute the full spans
span_x = 2.0 * a_half_x
span_y = 2.0 * a_half_y
span_z = 2.0 * a_half_z

# 4) Sort them: largest → length, next → width, smallest → thickness
length, width, height = sorted([span_x, span_y, span_z], reverse=True)

print(f"Length   : {length:.2f} mm")
print(f"Width    : {width:.2f} mm")
print(f"Height   : {height:.2f} mm")



Length   : 2280.01 mm
Width    : 203.76 mm
Height   : 203.20 mm


In [4]:
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib_Add
from OCC.Core.gp import gp_Pln, gp_Ax3, gp_Pnt, gp_Dir
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
import numpy as np

def compute_cross_section_area_occ(solid):
    # 1) Compute true volume centroid & long axis via inertia
    props = GProp_GProps()
    brepgprop.VolumeProperties(solid, props)
    cm = props.CentreOfMass()
    centroid = np.array([cm.X(), cm.Y(), cm.Z()])
    # inertia‐axis
    inertia = props.MatrixOfInertia()
    T = np.array([[inertia.Value(i,j) for j in (1,2,3)] for i in (1,2,3)])
    eigvals, eigvecs = np.linalg.eigh(T)
    long_idx = np.argmin(eigvals)
    long_dir = gp_Dir(*eigvecs[:, long_idx])

    # 2) Build infinite plane at centroid ⟂ long_axis
    plane = gp_Pln(gp_Ax3(gp_Pnt(*centroid), long_dir))

    # 3) Section the solid with that plane
    section = BRepAlgoAPI_Section(solid, plane)
    section.ComputePCurveOn1(True)
    section.Build()
    sect = section.Shape()

    # 4) Collect all edges into one wire
    wire_maker = BRepBuilderAPI_MakeWire()
    exp_edge = TopExp_Explorer(sect, TopAbs_EDGE)
    while exp_edge.More():
        wire_maker.Add(exp_edge.Current())
        exp_edge.Next()
    wire = wire_maker.Wire()

    # 5) Build a face from that wire
    face = BRepBuilderAPI_MakeFace(wire, True).Face()

    # 6) Compute exact face area
    props2 = GProp_GProps()
    brepgprop.SurfaceProperties(face, props2)
    area = props2.Mass()  # in mm²

    return area


In [12]:
import json
import numpy as np
import pandas as pd
from IPython.display import display

# 1) Compute cross-section area and estimated mass per meter
area = compute_cross_section_area_occ(solid)  # mm²
steel_density = 7850  # kg/m³
df_measured_mass = (area * 1e-6) * steel_density  # kg/m

# 2) Tolerances
tol_dim  = 1.0    # mm tolerance on dimensions
tol_csa  = 2.0    # % tolerance on CSA
tol_mass = 2.0    # % tolerance on mass

# 3) Load your updated JSON library with full beam details
with open("Shape_classifier_info.json") as f:
    profiles = json.load(f)

# 4) Identify section by comparing measured vs nominal properties
rows = []
for cat, ents in profiles.items():
    for name, info in ents.items():
        prof = f"{cat} {name}"
        nom_w   = info['width']
        nom_d   = info['height']
        nom_csa = info['csa']
        nom_mass= info['mass']

        errs = {}
        for orient, (w_meas, d_meas, w_nom, d_nom) in {
            'direct':  (width, height, nom_w, nom_d),
            'swapped': (width, height, nom_d, nom_w)
        }.items():
            err_w = abs(w_meas - w_nom)
            err_d = abs(d_meas - d_nom)
            if err_w <= tol_dim and err_d <= tol_dim:
                errs[orient] = (err_w / w_nom * 100, err_d / d_nom * 100)

        if not errs:
            continue

        orient, (ew, ed) = min(errs.items(), key=lambda kv: sum(kv[1]))
        err_csa  = abs(area - nom_csa) / nom_csa * 100
        err_mass = abs(df_measured_mass - nom_mass) / nom_mass * 100
        total_score = ew + ed + err_csa + err_mass

        rows.append({
            "Profile": prof,
            "Orientation": orient,
            "Err Width (%)": round(ew, 2),
            "Err Depth (%)": round(ed, 2),
            "Err CSA (%)": round(err_csa, 2),
            "Err Mass (%)": round(err_mass, 2),
            "Total Score": round(total_score, 2)
        })

# Build DataFrame and select prime candidate
df_identified_section = pd.DataFrame(rows).sort_values("Total Score").reset_index(drop=True)
prime = df_identified_section.iloc[0]

# Display results
print(f"Model Length : {length:.2f} mm")
print(f"Model Width  : {width:.2f} mm")
print(f"Model Height : {height:.2f} mm")
print(f"Area         : {area:.1f} mm²")
print(f"Measured kg/m: {df_measured_mass:.3f}")
display(df_identified_section)
print(f"Prime candidate: {prime['Profile']} ({prime['Orientation']}) with score {prime['Total Score']}%")

# 5) Extract DSTV header fields for the prime profile using updated JSON keys
cat, name = prime['Profile'].split(' ', 1)
prime_info = profiles[cat][name]

designation        = prime['Profile']
mass               = prime_info.get('mass')
height_val         = prime_info.get('height')
width_val          = prime_info.get('width')
csa                = prime_info.get('csa')
web_thickness      = prime_info.get('web_thickness')
flange_thickness   = prime_info.get('flange_thickness')
root_radius        = prime_info.get('root_radius')
toe_radius         = prime_info.get('toe_radius')
code_profile       = prime_info.get('code_profile')
length_val         = length  # from oriented bounding box

# 6) Store for DSTV header
header_dict = {
    "Designation":       designation,
    "Mass":              mass,
    "Height":            height_val,
    "Width":             width_val,
    "CSA":               csa,
    "web_thickness":     web_thickness,
    "flange_thickness":  flange_thickness,
    "root_radius":       root_radius,
    "toe_radius":        toe_radius,
    "code_profile":      code_profile,
    "Length":            length_val
}

print("\nDSTV Header Fields:", header_dict)

Model Length : 2280.00 mm
Model Width  : 203.60 mm
Model Height : 203.20 mm
Area         : 5873.1 mm²
Measured kg/m: 46.104


,Profile,Orientation,Err Width (%),Err Depth (%),Err CSA (%),Err Mass (%),Total Score
0,UC 203x203x46,direct,0.0,0.0,0.05,0.01,0.06


Prime candidate: UC 203x203x46 (direct) with score 0.06%

DSTV Header Fields: {'Designation': 'UC 203x203x46', 'Mass': 46.1, 'Height': 203.2, 'Width': 203.6, 'CSA': 5870.0, 'web_thickness': 7.2, 'flange_thickness': 11.0, 'root_radius': 10.2, 'toe_radius': 0.0, 'code_profile': 'I', 'Length': 2280.0}


In [46]:
import numpy as np
import pandas as pd
from IPython.display import display

from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAbs import GeomAbs_Cylinder
from OCC.Core.TopoDS import topods
from OCC.Core.gp import gp_Dir, gp_Pnt
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Display.SimpleGui import init_display
from OCC.Core.AIS import AIS_TextLabel
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB

# 1) Build centroid + local axes (L, F, W)
props = GProp_GProps(); brepgprop.VolumeProperties(solid, props)
cm = props.CentreOfMass()
centroid = np.array([cm.X(), cm.Y(), cm.Z()])

im = props.MatrixOfInertia()
T  = np.array([[im.Value(i,j) for j in (1,2,3)] for i in (1,2,3)])
ev, evec = np.linalg.eigh(T)
order = np.argsort(ev)  # [long, flange, web]
dirs = [gp_Dir(*evec[:,i]) for i in order]
L = np.array([dirs[0].X(), dirs[0].Y(), dirs[0].Z()])
F = np.array([dirs[1].X(), dirs[1].Y(), dirs[1].Z()])
# F = F / np.linalg.norm(F)
W = np.array([dirs[2].X(), dirs[2].Y(), dirs[2].Z()])

# 2) Compute half‐extents and origin
bary, half_extents, _ = get_oriented_boundingbox(solid)
spans = [2*h for h in half_extents]
length, width, thickness = sorted(spans, reverse=True)
origin = centroid - 0.5*length*L - 0.5*width*F - 0.5*thickness*W

# 3) Start the Qt viewer
viewer, start_viewer, _, _ = init_display()

# Beam in light gray:
gray = Quantity_Color(0.25, 0.25, 0.5, Quantity_TOC_RGB)
viewer.DisplayShape(solid, color=gray, transparency=0.5, update=False)

# 3) Classify & collect holes
tol_long = 0.7
tol_web  = 0.7
hole_data = []
face_id = 0

exp = TopExp_Explorer(solid, TopAbs_FACE)
while exp.More():
    face_id += 1
    face    = topods.Face(exp.Current())
    adaptor = BRepAdaptor_Surface(face, True)
    if adaptor.GetType() == GeomAbs_Cylinder:
        cyl = adaptor.Cylinder()
        xyzc = cyl.Axis().Direction().XYZ()
        g    = np.array([xyzc.X(), xyzc.Y(), xyzc.Z()])
        ng   = np.linalg.norm(g)

        # skip length holes
        if abs(np.dot(g, L))/(ng*np.linalg.norm(L)) > tol_long:
            exp.Next(); continue

        # centroid
        pf = GProp_GProps(); brepgprop.SurfaceProperties(face, pf)
        c  = pf.CentreOfMass()
        fc = np.array([c.X(), c.Y(), c.Z()])

        # classify V/O/U
        if abs(np.dot(g, W))/(ng*np.linalg.norm(W)) > tol_web:
            code, rgb = "V", (1.0, 0.0, 0.0)  # red
        else:
            side = np.dot(fc - centroid, F)
            if side > 0:
                code, rgb = "O", (0.0, 1.0, 0.0)  # green
            else:
                code, rgb = "U", (0.0, 0.0, 1.0)  # blue

        hole_data.append((face, code, rgb, fc))
        
    exp.Next()

# Align NC origin
a_half_L, a_half_F, a_half_W = half_extents
origin_nc1 = (
    centroid
  - a_half_L * L     # to the head cross‐section (H)
  - a_half_F * F     # to the outside flange face (O)
  + a_half_W * W     # now truly to the underside flange face (U)
)

# Build Hole Table
rows = []
for idx, (face, code, rgb, fc) in enumerate(hole_data, start=1):
    # 1) Diameter
    adaptor = BRepAdaptor_Surface(face, True)
    diam    = 2.0 * adaptor.Cylinder().Radius()

    # 2) Vector from NC1 origin to hole center
    v = fc - origin_nc1

    # 3) Conditional projection based on hole code
    #    V‐holes lie in the L–F plane; O/U‐holes in the L–W plane
    if code == "V":
        x = float(np.dot(v, L))   # along length
        y = float(np.dot(v, F))   # across flange
    elif code == "U":
        x = float(np.dot(v, L))   # along length
        y = -float(np.dot(v, W))   # through web (flange thickness)
    else:
        x2d = float(np.dot(v, L))
        y_raw = float(np.dot(v, W))
        y2d = 2*a_half_W - y_raw
    
    rows.append({
        "Hole #":        idx,
        "Code":          code,
        "Diameter (mm)": round(diam, 2),
        "X (mm)":        round(x, 0),
        "Y (mm)":        round(y, 0),
    })

# Build and display DataFrame
df_holes = pd.DataFrame(rows)
display(df_holes)

# Render each hole face & marker using Quantity_Color
for idx, (face, code, rgb, fc) in enumerate(hole_data, start=1):
    qcol = Quantity_Color(*rgb, Quantity_TOC_RGB)
    viewer.DisplayShape(face, color=qcol, update=False)
    
    # draw the sphere
    x, y, z = fc
    sph = BRepPrimAPI_MakeSphere(gp_Pnt(x, y, z), 5.0).Shape()
    viewer.DisplayShape(sph, color=qcol, update=False)

    # Add label to the sphere based on index (to match the table)
    label = AIS_TextLabel()
    label.SetText(f"{idx}")
    label.SetPosition(gp_Pnt(x, y, z))
    
    # black text for contrast
    label.SetColor(Quantity_Color(0.0, 0.0, 0.0, Quantity_TOC_RGB))

    viewer.Context.Display(label, False)

# Add the NC Origin
yellow = Quantity_Color(1.0, 1.0, 0.0, Quantity_TOC_RGB)
sphere = BRepPrimAPI_MakeSphere(gp_Pnt(*origin_nc1), 10.0).Shape()  # 10 mm radius
viewer.DisplayShape(sphere, color=yellow, update=True)


# Display Qt5 viewer
viewer.FitAll()
viewer.View_Iso()
start_viewer()


pyqt5 backend - Qt version 5.15.2


,Hole #,Code,Diameter (mm),X (mm),Y (mm)
0,1,V,12.0,2000.0,152.0
1,2,V,12.0,1500.0,152.0
2,3,V,12.0,900.0,152.0
3,4,V,12.0,300.0,152.0
4,5,V,12.0,300.0,52.0
5,6,V,12.0,900.0,52.0
6,7,V,12.0,1500.0,52.0
7,8,V,12.0,2000.0,52.0
8,9,U,12.0,425.0,50.0
9,10,O,12.0,425.0,50.0


Create 2d Drawing

>**Create DSTV**

In [45]:
# %%
"""
Jupyter Notebook: Write DSTV NC1 ST-block header to .nc1 file with specified order
"""
# Cell 1: Define DSTV header fields inline
# Replace placeholder values with your actual data
project_number     = 'PROJ-123'
out_filename        = '0444-1'
model_filename     = 'model.step'
material_grade     = 'S355JR'
quantity           = 1

faces = ['V', 'U', 'O']  #Face priority order

# %%
# Cell 2: Write the DSTV NC1 ST-block header to '0444-1.nc1'
filename = f"{out_filename}.nc1"
with open(filename, 'w') as f:
    f.write('ST\n')
    f.write(f"  Project Number\n")
    f.write(f"  Drawing Number\n")
    f.write(f"  {out_filename}\n")
    f.write(f"  {model_filename}\n")
    f.write(f"  {material_grade}\n")
    f.write(f"  {quantity}\n")
    f.write(f"  {header_dict['Designation']}\n")
    f.write(f"  {header_dict['code_profile']}\n")
    f.write(f"    {header_dict['Length']:8.2f}\n")
    f.write(f"    {header_dict['Height']:8.2f}\n")
    f.write(f"    {header_dict['Width']:8.2f}\n")
    f.write(f"    {header_dict['flange_thickness']:8.2f}\n")
    f.write(f"    {header_dict['web_thickness']:8.2f}\n")
    f.write(f"    {header_dict['root_radius']:8.2f}\n")
    f.write(f"    {header_dict['Mass']:8.2f}\n")
    f.write('        0.00\n') #surface area
    # Following the spec, three zeros
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('        0.00\n')
    f.write('  -\n')
    f.write('  -\n')
    f.write('  -\n')
    f.write('  -\n')
    
    # BO blocks by face
    for face in faces:
        df_face = df_holes[df_holes['Code'] == face]
        if df_face.empty:
            continue
        f.write('BO\n')
        for _, row in df_face.iterrows():
            x = row['X (mm)']
            y = row['Y (mm)']
            d = row['Diameter (mm)']
            # Right-align numeric columns: x, y (8-wide), diameter (6-wide)
            f.write(f"  {face.lower()}  {x:8.2f} {y:8.2f} {d:6.2f}\n")
            
    f.write('EN\n')
print(f"DSTV header written to {filename}")

DSTV header written to 0444-1.nc1
